<a href="https://colab.research.google.com/github/yrrSelena/RUL/blob/master/ANN_DE_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import time
import numpy as np
import csv
import sys
import pandas as pd
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from scipy.optimize import differential_evolution

Using TensorFlow backend.


In [0]:
from data_handler_CMAPSS import CMAPSSDataHandler
from tunable_model import SequenceTunableModelRegression
import aux_functions

In [0]:
#K.clear_session()  #Clear the previous tensorflow graph

l2_lambda_regularization = 0.20
l1_lambda_regularization = 0.10

def RULmodel_SN(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', 
                    kernel_regularizer=regularizers.L1L2(l1_lambda_regularization, l2_lambda_regularization), name='out'))
    
    return model

In [0]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = ''

window_size = 30
window_stride = 1
max_rul = 125

min_max_scaler = MinMaxScaler(feature_range=(0, 1))

dHandler_cmaps = CMAPSSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

In [0]:
def get_compiled_model(shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        """nFeatures = len(selected_features)
        shapeLSTM = (window_size, nFeatures)
        model = RULmodel_LSTM(shapeLSTM)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)"""
        pass
    else:
        pass

    return model

In [6]:
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = get_compiled_model(shapeSN, model_type='ann')
#SequenceTunableModelRegression(TunableModel)
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', modelRULLSTM, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_scaler = min_max_scaler

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
win_size = []
win_stride = []
const_rul = []
res_rmse_valid = []
res_rhs_valid = []
res_rmse_test = []
res_rhs_test = []

In [0]:
def nn_optmize_fun(x, tModel, verbose=0, epochs=250, saveToFile = None, iterations = 0):
    global win_size, win_stride, const_rul
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    maxWindowSize = {'1':30}#, '2':20, '3':30, '4':18}
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)

    #load the data using the selected parameters
    tModel.data_handler.sequence_length = x[0]
    #tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
    tModel.data_handler.sequence_stride = x[1]
    tModel.data_handler.max_rul = x[2]
    
    
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0.1, reload_data=True)
    
    if iterations == 0:
        print("Training model")
    
    
    #Create new model
    lrate = LearningRateScheduler(aux_functions.step_decay)
    nFeatures = len(selected_features)
    model_shape = nFeatures*tModel.data_handler.sequence_length
    modelRULSN = get_compiled_model(model_shape, model_type='ann')
    tModel.change_model('ModelRUL_SN_1', modelRULSN, 'keras')
    tModel.epochs = 20
    
    #Train model
    tModel.train_model(learningRate_scheduler=lrate, verbose=0)
    time = tModel.train_time
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluate_model(["rmse", "rhs"], cross_validation=True, round=2)
    cScores = tModel.scores
    rmse_valid = cScores['rmse']
    rhs_valid = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    #print(x)
    msgStr = "The model variables are " + str(x)# + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    print(msgStr)
    
    
    tModel.evaluate_model(["rmse", "rhs"], cross_validation = False, round=2)
    cScores = tModel.scores
    rmse_test = cScores['rmse']
    rhs_test = cScores['rhs']
    
    row = x.tolist() + [rmse_valid] +rhs_valid + [rmse_test] + rhs_test
    
    win_size.append(x[0])
    win_stride.append(x[1])
    const_rul.append(x[2])
    res_rmse_valid.append(rmse_valid)
    res_rhs_valid.append(rhs_valid)
    res_rmse_test.append(rmse_test)
    res_rhs_test.append(rhs_test)
    
    
    #print('------hhh---')
    if saveToFile is not None:
        #print(msgStr)
        writer = csv.writer(saveToFile)
        #row = x.append(rmse)
        #row = x.append(rhs)
        writer.writerow(row)
        #saveToFile.write(msgStr)
    else:
        print(row)
    
    #print('------hhh1---')
    #Return RMSE as the performance metric to steer the search
    return rmse_valid

In [0]:
csvfile = open("RUL_1_4_valid_res.csv", "w")
writer  = csv.writer(csvfile)
writer.writerow(win_size)
writer.writerow(win_stride)
writer.writerow(const_rul)

writer.writerow(res_rmse_valid)
writer.writerow(res_rhs_valid)
writer.writerow(res_rmse_test)
writer.writerow(res_rhs_test)
csvfile.close()

In [20]:
print(win_size)
print(win_stride)
print(const_rul)
print(res_rmse_valid)
print(res_rhs_valid)
print(res_rmse_test)
print(res_rhs_test)

[26, 6, 5, 25, 16, 18, 12, 1, 22, 13, 9, 24, 29, 3, 10, 25, 16, 18, 18, 20, 14, 21, 19, 24, 21, 15, 10, 21, 29, 18, 19, 20, 14, 19, 19, 21, 27, 28, 25, 21, 23, 28, 18, 20, 25, 22, 23, 19, 19, 27, 28, 21, 27, 26, 15, 21, 25, 25, 28, 24, 23, 14, 24, 26, 19, 20, 23, 21, 25, 20, 27, 29, 23, 23, 21, 26, 7, 29, 28, 13, 29, 20, 18, 18, 25, 25, 21, 23, 23, 29, 27, 22, 27, 23, 22, 28, 28, 1, 24, 28, 27, 27, 26, 29, 21, 24, 25, 21, 25, 27, 9, 20, 24, 27, 27, 24, 10, 27, 25, 21, 7, 20, 21, 24, 26, 27, 28, 22, 25, 24, 22, 24, 8, 2, 24, 21, 25, 22, 18, 24, 25, 28, 27, 23, 21, 20, 21, 18, 24, 19, 23, 2, 25, 8, 27, 21, 23, 27, 23, 25, 21, 25, 25, 19, 23, 21, 17, 21, 23, 25, 27, 26, 16, 19, 24, 21, 24, 22, 25, 20, 19, 27, 25, 26, 21, 12, 7, 25, 13, 22, 28, 29, 27, 7, 19, 25, 21, 27, 17, 23, 21, 21, 24, 21, 16, 29, 28, 29, 21, 22, 22, 21, 25, 10, 20, 26, 19, 27, 29, 26, 29, 27, 25, 27, 28, 21, 29, 29, 22, 20, 23, 26, 21, 25, 23, 26, 19, 18, 26, 21, 25, 25, 27, 26, 21, 22, 22, 2, 13, 18, 16, 24, 18, 21,

In [18]:
#Optimize the parameters for the NN using DE

#maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
maxWindowSize = {'1':30} #Do it only for datasets 1 and 2
totalTime = {'1':0}#, '2':0, '3':0, '4':0}
results = {'1':0}#, '2':0, '3':0, '4':0}

#datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']



for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("RUL_100_400_valid_res.csv", "w")

    windowSizeBounds = [1,30]
    windowStrideBounds = [1,10]
    constantRULBounds = [100,140]

    bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]
    #bounds = [windowStrideBounds, constantRULBounds] #Optimize only 2 variabes
    
    tModel.data_handler.change_dataset(datasetNumber)
    
    startTime = time.clock()
    
    tempResults = differential_evolution(nn_optmize_fun, bounds, 
                                     args=(tModel, 0, 20, file, 1),
                                    strategy='best1bin', maxiter=30, popsize=4, disp=True, polish=False)
    results[datasetNumber] = tempResults
    endTime = time.clock()

file.close()
totalTime[datasetNumber] = endTime - startTime

Tunning for dataset 1
Reloading data per user's request
Loading data for dataset 1 with window_size of 25, stride of 1 and maxRUL of 139. Cros-Validation ratio 0.1
Loading data from file and computing dataframes
training with cv
10/10 [==============================] - 0s 47us/step
The model variables are [ 25   1 139]
100/100 [==============================] - 0s 81us/step
Reloading data per user's request
Loading data for dataset 1 with window_size of 5, stride of 5 and maxRUL of 129. Cros-Validation ratio 0.1
Loading data from memory and recomputing dataframes
training with cv
10/10 [==============================] - 0s 47us/step
The model variables are [  5   5 129]
100/100 [==============================] - 0s 38us/step
Reloading data per user's request
Loading data for dataset 1 with window_size of 17, stride of 9 and maxRUL of 108. Cros-Validation ratio 0.1
Loading data from memory and recomputing dataframes
training with cv
10/10 [==============================] - 0s 47us/step
